# Transfer Learning with VGG and CIFAR-10 dataset
<hr>

## 1. Prepare & Explore Dataset

In [1]:
# import pachages 
from tensorflow import keras 
from keras.datasets import cifar10
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical 

from keras.applications import VGG19

from keras.optimizers import SGD,Adam

In [2]:
# load data.
(x_train,y_train),(x_test,y_test)=cifar10.load_data()

In [3]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.3)

In [4]:
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((35000, 32, 32, 3), (35000, 1))
((15000, 32, 32, 3), (15000, 1))
((10000, 32, 32, 3), (10000, 1))


In [5]:
# We have 10 classes, so, our network will have 10 output neurons
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test) 

In [6]:
print((x_train.shape,y_train.shape))
print((x_val.shape,y_val.shape))
print((x_test.shape,y_test.shape))

((35000, 32, 32, 3), (35000, 10))
((15000, 32, 32, 3), (15000, 10))
((10000, 32, 32, 3), (10000, 10))


We can now begin the actual process of model building.
The following a set process and following consistently makes learning this easier :

- Define the Data Augmentation (ImageDataGenerator) 
- Build the model (Base Model + Flatten + Dense)
- Check model summary
- Initialize Batch Size,Number of Epochs
- Compile model
- Fit the model 
- Evaluate the model 

In [7]:
#Data Augmentation Function: Let's define an instance of the ImageDataGenerator class and set the parameters.
#We have to instantiate for the Train,Validation and Test datasets
train_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1 )

val_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip=True,
                                    zoom_range=.1)

test_generator = ImageDataGenerator(
                                    rotation_range=2, 
                                    horizontal_flip= True,
                                    zoom_range=.1) 

## 2. Define the neural network architecture

In [8]:
# define the CNN model
'The first base model used is VGG19. The pretrained weights from the imagenet challenge are used'
base_model_1 = VGG19(include_top=False, weights='imagenet', input_shape=(32,32,3), classes=y_train.shape[1])

In [9]:
#Lets add the final layers to these base models where the actual classification is done in the dense layers
model_1 = Sequential()
model_1.add(base_model_1) 
model_1.add(Flatten()) 

In [10]:
print(model_1.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
Total params: 20,024,384
Trainable params: 20,024,384
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
#Add the Dense layers along with activation and batch normalization
model_1.add(Dense(1024,activation=('relu'),input_dim=512))
model_1.add(Dense(512,activation=('relu'))) 
model_1.add(Dense(256,activation=('relu'))) 
model_1.add(Dropout(.3))#Adding a dropout layer that will randomly drop 30% of the weights
model_1.add(Dense(128,activation=('relu')))
model_1.add(Dropout(.2))
model_1.add(Dense(10,activation=('softmax'))) #This is the classification layer

In [12]:
#Check final model summary
model_1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg19 (Functional)           (None, 1, 1, 512)         20024384  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 1024)              525312    
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               3

## 3. Compile the neural net

In [13]:
# compile your model
model_1.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

## 4. Fit / train the neural net

In [14]:
batch_size= 100
epochs=20

model_1.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val), verbose=1) 

Epoch 1/50
350/350 [==============================] - 1167s 3s/step - loss: 2.4855 - accuracy: 0.1139 - val_loss: 2.2419 - val_accuracy: 0.1344
Epoch 2/50
350/350 [==============================] - 1142s 3s/step - loss: 2.0888 - accuracy: 0.1926 - val_loss: 1.5159 - val_accuracy: 0.4477
Epoch 3/50
350/350 [==============================] - 1122s 3s/step - loss: 1.5009 - accuracy: 0.4750 - val_loss: 0.9628 - val_accuracy: 0.6746
Epoch 4/50
350/350 [==============================] - 1097s 3s/step - loss: 1.0183 - accuracy: 0.6692 - val_loss: 0.7935 - val_accuracy: 0.7381
Epoch 5/50
350/350 [==============================] - 1103s 3s/step - loss: 0.7859 - accuracy: 0.7443 - val_loss: 0.6833 - val_accuracy: 0.7732
Epoch 6/50
350/350 [==============================] - 1091s 3s/step - loss: 0.6350 - accuracy: 0.7951 - val_loss: 0.6362 - val_accuracy: 0.7923
Epoch 7/50
350/350 [==============================] - 1084s 3s/step - loss: 0.5379 - accuracy: 0.8264 - val_loss: 0.6473 - val_accuracy:

KeyboardInterrupt: 

## 5. Evaluate the neural net

In [16]:
score = model_1.evaluate(x_val, y_val, verbose=1) 
print('Test accuracy:', score[1]) 

469/469 [==============================] - 33s 69ms/step - loss: 1.3226 - accuracy: 0.8456
Test accuracy: 0.8456000089645386


## 6. Make predictions / classifications for unseen data

In [18]:
#not yet until we enhanced the results
predictions = model_1.predict(x_test)
predictions

array([[1.06216369e-09, 1.93122504e-10, 1.44084011e-09, ...,
        3.27734062e-09, 3.08773146e-10, 3.88189814e-10],
       [8.52375184e-11, 6.95966804e-08, 4.45967167e-15, ...,
        3.74803113e-17, 9.99999881e-01, 2.36915106e-11],
       [9.32551302e-07, 2.81470534e-06, 4.51360421e-10, ...,
        2.67097334e-11, 9.99996185e-01, 1.12571456e-07],
       ...,
       [8.69667700e-15, 9.02202109e-16, 1.02165706e-10, ...,
        8.57347873e-12, 1.04837193e-13, 8.09200909e-16],
       [2.24883550e-07, 9.99997973e-01, 9.96809835e-09, ...,
        1.05066600e-09, 1.47253229e-06, 3.53377743e-07],
       [4.67092023e-13, 6.94539806e-16, 2.71848164e-13, ...,
        1.00000000e+00, 4.50566841e-16, 6.64202200e-13]], dtype=float32)

<hr>